<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex03notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex03notebookB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 識別のための教師あり学習(2) 最近傍法と $k$-近傍法
----




----
### 準備


以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
### 例題: 人間 vs ほげ星人

識別の問題について考える際の具体例として，「身長(cm)」と「体重(kg)」という二つの値から，「人間」と「ほげ星人」を識別する問題を考えます．



In [ ]:
# 「人間 vs ほげ星人」データの入手
dfHoge = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/humanvshoge.csv', header=0)

データの中身はこんなん：

In [ ]:
dfHoge

1行がひとりぶんのデータ．'height' が身長，'weight'が体重，'label' が 'Human' なのは人間，'Hoge' なのはほげ星人，ということです．

In [ ]:
# 学習データのうち (身長, 体重) の値を配列 X へ，クラスラベルを配列 Y へ
X = dfHoge.loc[:, ['height', 'weight']]
Y = dfHoge['label']
X_human = X[Y=='Human'].to_numpy()
X_hoge  = X[Y=='Hoge'].to_numpy()

In [ ]:
# 身長を横軸に，体重を縦軸にとって散布図を描く
# 人間が青でほげ星人がオレンジ
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.set_xlim(0,250)
ax.set_xlabel('height [cm]')
ax.set_ylim(0,150)
ax.set_ylabel('weight [kg]')
ax.set_aspect('equal')
ax.scatter(X_human[:, 0], X_human[:, 1])
ax.scatter(X_hoge[:, 0], X_hoge[:, 1])
plt.show()

ひとりひとりが $(身長,体重)$ という二つの数値の組で表されていますので，これら二つの値で散布図を描いてみると，ひとりが一つの点に対応します．
図の青い点が人間，オレンジの点がほげ星人（注）．

<span style="font-size: 75%">
※注: ここで扱っているデータは，架空のものです．説明を簡単にするために身長や体重の値を適当に設定しています．例題としての扱いを超えて，「身長や体重の値が〇〇だと人間ではない」というような主張をしているわけではありません．
</span>

----
### 最近傍法とは


**最近傍法** (Nearest Neighbor Method)は，識別の方法の一つです．
**最短距離法** と同様に，データとデータの間の距離（注）に基づいて未知データのクラスを決定します．

<span style="font-size: 75%">
※注: 距離の規準としては，一般的なユークリッド距離の他に，データの性質に応じて様々なものが用いられます．
</span>

最短距離法の手順は，こんなんでした：

1. クラスごとに見本となるデータ（**プロトタイプ** (prototype)）を用意しておく．
1. 所属クラスが未知のデータが与えられたら，そのデータがどのプロトタイプと近いかを調べる．
1. 一番近いプロトタイプと同じクラスに分類する．


実は，上記の手順は最近傍法の手順にもなっています．ただし，最短距離法ではプロトタイプを一クラス一つしか用いませんでしたが，最近傍法では複数のプロトタイプを用います．
一般的には，学習データとして用意されたデータを全てプロトタイプとして扱います．


以下に「人間 vs ほげ星人」の識別を最近傍法で行うプログラムを用意しました．学習データを全てプロトタイプとし，距離はユークリッド距離で測ります．

In [ ]:
# 「人間 vs ほげ星人」最近傍法による識別実験のための関数
#
def hoge(X, Y, height, weight):

    # 学習データ
    X_human = X[Y=='Human'].to_numpy()
    X_hoge  = X[Y=='Hoge'].to_numpy()

    # 識別対象の身長と体重
    x = np.array([height, weight])

    # 最近傍のデータを見つける
    d2 = np.sum((X - x)**2, axis=1) # 各データとの距離の2乗
    imin = np.argmin(d2)  # 距離最小のデータの番号を求める
    p_h, p_w, p_lab = X.iloc[imin, 0], X.iloc[imin, 1], Y.iloc[imin]
    print(f'({x[0]}, {x[1]}) との距離が最小なのは{imin}番:({p_h}, {p_w})  クラスラベルは {p_lab}')

    # グラフを描く
    fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
    ax.set_xlim(0,250)
    ax.set_xlabel('height [cm]')
    ax.set_ylim(0,150)
    ax.set_ylabel('weight [kg]')
    ax.set_aspect('equal')
    # 学習データの点を描く
    ax.scatter(X_human[:, 0], X_human[:, 1])
    ax.scatter(X_hoge[:, 0], X_hoge[:, 1])
    # 識別対象の点を描く
    ax.plot(x[0], x[1], marker='*', markersize=20, color='green')
    # 識別対象とその最近傍の点の間に線分を引く
    ax.plot([x[0], X.iloc[imin, 0]], [x[1], X.iloc[imin, 1]], linestyle='-', color='gray')
    plt.show()


In [ ]:
#@title height，weightの値をいろいろ変えて実験しよう（値を変えたらセルの再実行忘れずに）
height = 140.0 #@param {type:"number"}
weight = 100 #@param {type:"number"}

# (height, weight) のひとはどっち？
hoge(X, Y, height, weight)

#### ★★ やってみよう ★★
1. $(\textrm{height}, \textrm{weight})$ の値をいろいろ変えて上記のセルを実行し，結果を観察しよう．
1. 以下のひとは人間と識別されるかほげ星人と識別されるかセルを実行して確認しよう．
結果をノート等（紙媒体）にメモしておこう．
    - $(身長, 体重) = (150, 100)$ のひと
    - $(身長, 体重) = (130, 75)$ のひと
    - $(身長, 体重) = (130, 78)$ のひと

----
### 最近傍法の性質



最短距離法では，二つのクラスを分ける識別境界は，2つのプロトタイプの垂直二等分線（より高次元では同様の平面）になるのでした．
最近傍法の場合は，一つのクラスに複数のプロトタイプがあるため，識別境界はもっと複雑になります．

下図左のような2次元3クラス（点の色がクラスを表します）のデータが与えられたときに，これらをプロトタイプとして，この平面上の各点を最近傍法で3クラスに分類して色を塗り分けると，下図右のようになります．
プロトタイプの配置に応じて入り組んだ識別境界ができているのがわかりますね．

<img width="75%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/2dim3class3.png">

また，最近傍法を大規模なデータに適用したい場合は，その計算コスト（計算にかかる手間）にも気を配らねばなりません．
機械学習では学習データの数が多い方がよい結果を得られやすいので，例えば画像を扱うような場合でも，その数が数万とか数百万とかになり得ます．このような大量のデータをすべてプロトタイプとして最近傍法を実行しようとすると，一つのデータの識別結果を得るために膨大な数のプロトタイプとの距離を計算することになり，現実的な時間で結果が得られなくなったりします（注）．

<span style="font-size: 75%">
※注: 大規模なデータに対して最近傍法を適用したい場合，あらかじめ学習データの中からプロトタイプを選別するとか，最も距離の小さいデータを見つける処理（最近傍探索といいます）を効率よくまたは近似的に行う高速計算アルゴリズムを採用するとかします．
</span>

----
### $k$-近傍法

**最近傍法** を拡張した識別手法に，**$k$-近傍法** ($k$-Nearest Neighbor Method)というものがあります．これは，次のようなものです．

1. クラスごとに見本となるデータ（これを **プロトタイプ** (prototype）といいます）を用意しておく．
1. 所属クラスが未知のデータが与えられたら，そのデータとの距離が小さい方から $k$ 個のプロトタイプを見つける．
1. これらプロトタイプの所属クラスの多数決によって未知データの所属クラスを決定する．

この説明からわかると思いますが，$k = 1$ の$k$-近傍法（$1$-近傍法）は最近傍法そのものです．
下図に，2次元2クラスのデータ（赤と青の点）を用いて，$1$-近傍法と$7$-近傍法のそれぞれで平面を赤と青の2クラスに塗り分けたものを示します．
$k=7$の方では，平面上の各点に入った赤青の票の数に応じて色の濃さを変えてあります．
$k$ を大きくすることで，識別境界（赤と青の境目）が滑らかな形になっていることがわかります．

<img width="75%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/knn.png">

この例でもわかるように，$k$-近傍法による識別の結果は $k$の値によって変わります．ですので，この $k$ もある意味パラメータのようなものです．
しかし，一般的なパラメータと違い，学習の過程で自動的に決まったりはしません．
通常は，人間があらかじめ適当に決めておきます．このように，機械学習モデルの中には，自動的に調節できないパラメータが存在する場合があります．
このようなパラメータのことを，**ハイパーパラメータ** (Hyper Parameter) といいます．

自動的に決められないとはいえ，ハイパーパラメータをいい加減に決めるのはよくありません．
その辺りの話は，「機械学習II」の方で少し説明する予定です．

----
### 例題: 手書き数字の識別

0 から 9 までの手書き数字の画像から，その画像に写っている数がいくつかを答えさせる問題を考えます．

ここで扱うデータは，機械学習の分野で超有名な [MNIST](http://yann.lecun.com/exdb/mnist/) と呼ばれるデータセットからとったものです．
MNIST のデータは学習用だけで6万枚の画像がありますので，そこからランダムに一部の画像を抽出したものを用意しました．


In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数
D = datL.shape[1] # データの次元数 28 x 28 = 784

以下のセルでは関数を定義しています．定義してるだけですので，このセルを実行しただけでは何も起こりません．

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

学習データ中の最初の50個の画像を表示．

In [ ]:
nx, ny = 10, 5
display(datL[:50], nx, ny)

それぞれの正解クラスを表示．

In [ ]:
for iy in range(ny):
    print(labL[iy*nx:(iy+1)*nx])

----
### 最近傍法による手書き数字の識別

上記のデータに対して最近傍法を適用して識別させてみましょう．
学習データすべてをプロトタイプとし，距離はユークリッド距離で測ることにします．


まずは，学習データを識別させてみます．
ただし本当は，学習データ全部をプロトタイプとした最近傍法の場合，学習データに対する識別はやってみるまでもありません．やらなくてもどういう結果になるか，識別率がいくつかはわかってます．どうしてか考えてみてね．

次のセル，実行に少し時間がかかります．先に説明した通り，距離の計算をたくさんやってるためです（注）．

<span style="font-size: 75%">
※注: 実は，高校数学（ベクトルの性質）と少しのプログラミングの知識でもう少し効率のよい計算法を実装できるのですが，その辺はおまけ課題にする...かも．
</span>

In [ ]:
# 学習データを識別
N = len(datL)
ncorrect = 0
for n in range(N):
    if n % 500 == 0:
        print(f'{n}/{N}')
    dist2 = np.sum((datL[n, :] - datL)**2, axis=1) # 各プロトタイプとの距離の2乗
    out = labL[np.argmin(dist2)]  # 識別されたクラス
    if out == labL[n]:  # 正解数をカウント
        ncorrect += 1

print(f'識別率: {ncorrect}/{N} = {ncorrect/N}')

次は，学習データとは別に用意したデータ（以下「テストデータ」と呼ぶことにします）を識別させてみます．

In [ ]:
# テストデータを識別
N = len(datT)
out = np.empty(N, dtype=int)
for n in range(N):
    dist2 = np.sum((datT[n, :] - datL)**2, axis=1) # 各プロトタイプとの距離の2乗
    out[n] = labL[np.argmin(dist2)]  # 識別されたクラス

ncorrect = np.sum(out == labT) # 正解数をカウント

print(f'識別率: {ncorrect}/{N} = {ncorrect/N}')

テストデータのうちいくつかについて，最も距離が小さいと判断されたプロトタイプの画像とそのクラスラベルを表示してみると...


In [ ]:
# datT の中からいくつか選択して識別結果を表示
idx = np.array([5, 6, 7, 8, 9, 35, 36, 37, 38, 39])
dat = datT[idx, :]
N = len(dat)
for n in range(N):
    dist2 = np.sum((dat[n, :] - datL)**2, axis=1) # 各プロトタイプとの距離の2乗
    out[n] = np.argmin(dist2)  # 距離最小のプロトタイプの番号

display(dat, N, 1)
print('上記の画像の正解クラス番号:', labT[idx])
display(datL[out[:N], :], N, 1)
print('最も近かったプロトタイプの画像とそのクラス番号:', labL[out[:N]])

#### ★★ やってみよう ★★

1. 上記の実験の以下の記述のところ．考えてみてね．
> ただし本当は，学習データ全部をプロトタイプとした最近傍法の場合，学習データに対する識別はやってみるまでもありません．やらなくてもどういう結果になるか，識別率がいくつかはわかってます．どうしてか考えてみてね．
1. 上記の実験では，テストデータに対する識別率はいくつだったか．メモしておこう．
1. 上記の実験では，正解のクラスが
    > [6 9 7 9 1 6 0 5 7 6]

    である10枚の画像たちを最近傍法で識別させた結果を表示しています．
そのうち，識別結果が間違っていたものについて，「正解のクラスは〇だったが間違って●と識別していた」のようにメモしておきましょう．
